In [ ]:
import piplite
await piplite.install("numpy")
await piplite.install("matplotlib")

<br><br><br>

## Orbitty

In [ ]:
from my_cool_package import orbitty

<br><br><br>

This is a little simulation of gravitational forces.

In [ ]:
system = orbitty.System(m=[1, 1], x=[[0, -1, 0], [0, 1, 0]], p=[[1, 0, 0], [-1, 0, 0]])
system.steps(300, dt=0.1)
system.plot()

The number of dimensions matters: only 3D positions and momenta have an inverse-square law.

In [ ]:
system = orbitty.System(m=[1, 1], x=[[0, -1], [0, 1]], p=[[1, 0], [-1, 0]])
system.steps(1500, dt=0.01)
system.plot()

A comet!

In [ ]:
system = orbitty.System(
    m=[1000, 1], x=[[0, 0, 0], [0, 10, 0]], p=[[10, 0, 0], [-10, 0, 0]]
)
system.steps(300)
system.plot()

A moon!

In [ ]:
system = orbitty.System(
    m=[1000, 1, 1],
    x=[[0, 0, 0], [0, 10, 0], [0, 11, 0]],
    p=[[0, 0, 0], [-16, 0, 0], [-13, 0, 0]],
)
system.steps(600)
system.plot()

The three-body problem!

In [ ]:
p1 = 0.347111
p2 = 0.532728
system = orbitty.System(
    m=[1, 1, 1],
    x=[[-1, 0, 0], [1, 0, 0], [0, 0, 0]],
    p=[[p1, p2, 0], [p1, p2, 0], [-2 * p1, -2 * p2, 0]],
)
system.G = 1
system.steps(1000, dt=0.01)
system.plot()

A whole lot of particles!

In [ ]:
system = orbitty.System.random(
    num_particles=20,
    num_dimensions=3,
    mass_mean=10,
    mass_width=1,
    x_width=100,
    p_width=10,
)
system.steps(10000, dt=0.1)
system.plot()

A whole-whole lot of particles!

In [ ]:
system = orbitty.System.random(
    num_particles=50,
    num_dimensions=3,
    mass_mean=10,
    mass_width=1,
    x_width=100,
    p_width=15,
)
system.steps(10000, dt=0.1)
system.plot()

<br><br><br>

## What would be a good unit test?

It's hard to test a plot.

Even if you get the plot as an image or a video file and test it against an expected image/video,

* you'd have to compare the _uncompressed_ image/video, since minor differences in codec versions can make the compressed bytes differ when nothing has really changed,
* the test might be run on a computer with different plotting backend, making the plots differ in irrelevant ways (margins, fonts, ...),
* if you ever want to add tests, you'll have to go through a complicated process of making expected images/videos.

It's not worth it!

<br><br><br>

What about testing raw values?

In [ ]:
system = orbitty.System(
    m=[1000, 1], x=[[0, 0, 0], [0, 10, 0]], p=[[10, 0, 0], [-10, 0, 0]]
)
system.steps(1000)

In [ ]:
system.t_history

In [ ]:
system.x_history

In [ ]:
system.p_history

Again, that's a lot of precise values that you'd have to store somewhere.

If making tests is not fun & easy, you're not going to want to do it.

<br><br><br>

**Physics question:** what are the invariants of motion for this physical system?

In [ ]:
import numpy as np

### One

In [ ]:
def total_momentum(system):
    return np.sum(system.p, axis=0).tolist()

### Two

In [ ]:
def total_energy(system):
    # KE = 1/2 m |v|^2 and p = mv, so KE = 1/2 |p|^2 / m
    kinetic = np.sum(0.5 * np.sum(system.p**2, axis=1) / system.m)

    # gravitational force -> potential integration in 3D
    assert system.num_dimensions == 3
    # indexes to pick out (particle 1, particle 2) pairs, for all pairs
    p1, p2 = np.triu_indices(len(system.x), k=1)
    # pairwise (pw) displacements between all particle pairs
    pw_displacement = system.x[p2] - system.x[p1]
    # pairwise displacement is a sum in quadrature over all dimensions
    pw_distance = np.sqrt(np.sum(pw_displacement**2, axis=-1))
    # PE = -G m1 m2 / distance (for each pair)
    pw_potential = -system.G * system.m[p1] * system.m[p2] / pw_distance
    # sum over pairs to get the potential for each particle
    particle_potential = np.zeros_like(system.m)
    np.add.at(particle_potential, p1, pw_potential)
    np.add.at(particle_potential, p2, pw_potential)
    # avoid double-counting (particle 1, particle 2) and (particle 2, particle 1)
    potential = 0.5 * np.sum(particle_potential)

    return potential + kinetic

### Three

In [ ]:
def center_of_mass(system):
    return np.sum(
        system.m[:, np.newaxis] * system.x / np.sum(system.m), axis=0
    ).tolist()

will move linearly.

If the initial momentum is zero, then this is a constant.

### More?

There's also Kepler's laws of motion, but they're equivalent to the above.

### Applying them

Center of mass on the comet.

In [ ]:
system = orbitty.System(
    m=[1000, 1], x=[[0, 0, 0], [0, 10, 0]], p=[[10, 0, 0], [-10, 0, 0]]
)

initial = center_of_mass(system)
for i in range(1000):
    system.step()
    assert center_of_mass(system) == initial, f"{i}\n{center_of_mass(system)}\n{initial}"

Perfection is too much to ask of a numerical simulation.

In [ ]:
import pytest

In [ ]:
system = orbitty.System(
    m=[1000, 1], x=[[0, 0, 0], [0, 10, 0]], p=[[10, 0, 0], [-10, 0, 0]]
)

initial = center_of_mass(system)
for i in range(1000):
    system.step()
    assert center_of_mass(system) == pytest.approx(
        initial
    ), f"{i}\n{center_of_mass(system)}\n{initial}"

<br><br><br>

Total momentum on the moon.

In [ ]:
system = orbitty.System(
    m=[1000, 1, 1],
    x=[[0, 0, 0], [0, 10, 0], [0, 11, 0]],
    p=[[0, 0, 0], [-16, 0, 0], [-13, 0, 0]],
)

initial = total_momentum(system)
for i in range(1000):
    system.step()
    assert total_momentum(system) == pytest.approx(
        initial
    ), f"{i}\n{total_momentum(system)}\n{initial}"

<br><br><br>

Total energy on the three-body problem.

In [ ]:
p1 = 0.347111
p2 = 0.532728
system = orbitty.System(
    m=[1, 1, 1],
    x=[[-1, 0, 0], [1, 0, 0], [0, 0, 0]],
    p=[[p1, p2, 0], [p1, p2, 0], [-2*p1, -2*p2, 0]]
)
system.G = 1

initial = total_energy(system)
for i in range(1000):
    system.step()
    assert total_energy(system) == pytest.approx(initial), f"{i}\n{total_energy(system)}\n{initial}"

Hmmm. That's really close.

Let's try loosening the tolerance on [pytest.approx](https://docs.pytest.org/en/latest/reference/reference.html#pytest-approx).

In [ ]:
p1 = 0.347111
p2 = 0.532728
system = orbitty.System(
    m=[1, 1, 1],
    x=[[-1, 0, 0], [1, 0, 0], [0, 0, 0]],
    p=[[p1, p2, 0], [p1, p2, 0], [-2 * p1, -2 * p2, 0]],
)
system.G = 1

initial = total_energy(system)
for i in range(1000):
    system.step()
    assert total_energy(system) == pytest.approx(
        initial, rel=1e-2
    ), f"{i}\n{total_energy(system)}\n{initial}"

Is that good enough?

Remember that automated tests are _for your benefit_. The question is: what would convince _you_ that there's truly something wrong and not some round-off error?

What about reducing the step size in the numerical simulation, instead of widening the tolerance for error?

<br><br><br>

### Fuzz tests

These are all canned examples, with carefully chosen initial conditions, masses that are either 1 or 1000, and zero components in the third dimension. That's not really sampling the space of possibilities. What if a user tries something unexpected?

Maybe we need some randomly generated tests.

In [ ]:
system = orbitty.System.random(
    num_particles=20,
    num_dimensions=3,
    mass_mean=10,
    mass_width=1,
    x_width=100,
    p_width=10,
)

initial = total_momentum(system)
for i in range(10000):
    system.step()
    assert total_momentum(system) == pytest.approx(
        initial
    ), f"{i}\n{total_momentum(system)}\n{initial}"

Out of curiosity, what did that simulation look like?

In [ ]:
system.plot()

The fact that the total momentum remained constant, while the particles all weave around each other in 3D, is an impressive demonstration that it's correct.

But... there's always some numerical error, hidden by the tolerances in [pytest.approx](https://docs.pytest.org/en/latest/reference/reference.html#pytest-approx).

What if one run of the test just happens to pick a random number that scrapes the boundary of pytest's tolerances? The test will fail, and if we try to run it again, it probably won't fail again. If the failure happened on GitHub Actions, it might leave us wondering whether there's a system-dependent error in our code!

Non-reproducible bugs are the worst!

<br><br><br>

Don't let your tests be non-deterministic, even if they're random.

In [ ]:
rng = np.random.default_rng(seed=12345)

rng.integers(0, 100, 10)

In [ ]:
rng = np.random.default_rng(seed=12345)

system = orbitty.System.random(
    num_particles=20,
    num_dimensions=3,
    mass_mean=10,
    mass_width=1,
    x_width=100,
    p_width=10,
    rng=rng,
)

initial = total_momentum(system)
for i in range(10000):
    system.step()
    assert total_momentum(system) == pytest.approx(
        initial
    ), f"{i}\n{total_momentum(system)}\n{initial}"

It's "random" in the sense that this scenario was randomly chosen _once_, but then every time we run it, we start from the same scenario.

If anything glitchy happens on a the test-runner, we'll know that it's not variations in the random seed. When debugging, you'll need to have as few sources of error as possible. Don't let an unseeded random number generator add one more.

<br><br><br>

### Also, include a simple test

Although these invariants are general and physics-motivated, what if one of them fails? What if they all fail? What's really happening in the code?

What if we run

In [ ]:
rng = np.random.default_rng(seed=12345)

system = orbitty.System.random(
    num_particles=5,
    num_dimensions=3,
    mass_mean=10,
    mass_width=1,
    x_width=100,
    p_width=10,
    rng=rng,
)
system.steps(10)

and instead of

In [ ]:
system.t_history

In [ ]:
system.x_history

In [ ]:
system.p_history

we get subtly different values?

What if we get radically different values?

In both cases, the physics invariants will fail, so what's the next diagnostic step?

<br><br><br>

If we also save

In [ ]:
expected_t_history = system.t_history
expected_x_history = system.x_history
expected_p_history = system.p_history

in a file and load them in a test like

In [ ]:
rng = np.random.default_rng(seed=12345)

system = orbitty.System.random(
    num_particles=5,
    num_dimensions=3,
    mass_mean=10,
    mass_width=1,
    x_width=100,
    p_width=10,
    rng=rng,
)
system.steps(10)

assert system.t_history.tolist() == pytest.approx(expected_t_history)
assert system.x_history.tolist() == pytest.approx(expected_x_history)
assert system.p_history.tolist() == pytest.approx(expected_p_history)

then we'll be able to distinguish between bugs in which `expected_x_history` is off by 0.001 from bugs in which `expected_x_history` is off by 1e12.

Having at least one of these will make your debugging easier.